## This script gets the statisctical informaiton about predictions

In [1]:
######### import packages
import pandas as pd
import numpy as np
import os
import glob
import torch

Below, we start to run our script. Please note that the folders name in this script should be modified and set according to what model folders you have.

In [12]:
if __name__ == '__main__':
    
    #------------ set up parameters
    layer_num = [2, 5, 10]
    hidden_dim = [32]
    # the directory of test results
    # "DNN_Models/DNN_Regression/deepnn_results/test_results"
    root_dir = os.path.join('../','src','DNN_Regression','deepnn_results','test_results')
    
    # to save the statistical results
    stat_info_dict = {'Layer_num':[],
                     'Hidden_dim':[],
                     'Trn_RMSE':[],
                     'Val_RMSE':[],
                     'Tst_RMSE':[],
                     'Pred_-12_RMSE':[],
                     'Pred_-12_mean':[],
                     'Pred_-12_std':[],
                     'Pred_-29_RMSE':[],
                     'Pred_-29_mean':[],
                     'Pred_-29_std':[],
                     'Pred_-41_RMSE':[],
                     'Pred_-41_mean':[],
                     'Pred_-41_std':[],
                     }
    
    for cur_layer in layer_num:
        for cur_hd in hidden_dim:
            # predictions of training set
            trn_pred_file = os.path.join(root_dir, 'train_prediction_L{}_H{}.csv'.format(cur_layer, cur_hd))
            val_pred_file = os.path.join(root_dir, 'validation_prediction_L{}_H{}.csv'.format(cur_layer, cur_hd))
            tst_pred_file = os.path.join(root_dir, 'hos_test_prediction_L{}_H{}.csv'.format(cur_layer, cur_hd))
            rmse_file = os.path.join(root_dir, 'final_RMSE_L{}_H{}.csv'.format(cur_layer, cur_hd))
            
            # read data to dataframe
            trn_pred_df = pd.read_csv(trn_pred_file)
            val_pred_df = pd.read_csv(val_pred_file)
            tst_pred_df = pd.read_csv(tst_pred_file)
            rmse_df = pd.read_csv(rmse_file)
            trn_rmse = rmse_df[rmse_df['Type']=='Train_RMSE']['RMSE'].values[0]
            val_rmse = rmse_df[rmse_df['Type']=='Validation_RMSE']['RMSE'].values[0]
            tst_rmse = rmse_df[rmse_df['Type']=='Test_RMSE']['RMSE'].values[0]
            
           
            held_out_positions = []
            held_out_positions = tst_pred_df['Y_True'].unique()
            print(held_out_positions)
            
            # process each held_out positon:
            temp_dict = {'-12':[], '-29':[], '-41':[]}
            
            for cur_position in held_out_positions:
                cur_pred_df = tst_pred_df[tst_pred_df['Y_True']==cur_position]
                cur_Y_true = cur_pred_df['Y_True'].values
                cur_Y_true = torch.from_numpy(cur_Y_true)
                cur_Y_pred = cur_pred_df['Y_Prediction'].values
                cur_Y_pred = torch.from_numpy(cur_Y_pred)
                loss = torch.nn.MSELoss()
                cur_mse = loss(cur_Y_pred, cur_Y_true)
                cur_mse = torch.sqrt(cur_mse) #RMSE
                cur_mse = cur_mse.item()
                cur_pred_mean = cur_pred_df['Y_Prediction'].mean()
                cur_pred_std = cur_pred_df['Y_Prediction'].std()
                
                cur_position = str(cur_position).split('.')[0]
                temp_dict[cur_position].append([cur_mse, cur_pred_mean, cur_pred_std])
                
                
           
            stat_info_dict['Layer_num'].append(cur_layer)
            stat_info_dict['Hidden_dim'].append(cur_hd)
            stat_info_dict['Trn_RMSE'].append(trn_rmse)
            stat_info_dict['Val_RMSE'].append(val_rmse)
            stat_info_dict['Tst_RMSE'].append(tst_rmse)
            stat_info_dict['Pred_-12_RMSE'].append(temp_dict['-12'][0][0])
            stat_info_dict['Pred_-12_mean'].append(temp_dict['-12'][0][1])
            stat_info_dict['Pred_-12_std'].append(temp_dict['-12'][0][2])
            stat_info_dict['Pred_-29_RMSE'].append(temp_dict['-29'][0][0])
            stat_info_dict['Pred_-29_mean'].append(temp_dict['-29'][0][1])
            stat_info_dict['Pred_-29_std'].append(temp_dict['-29'][0][2])
            stat_info_dict['Pred_-41_RMSE'].append(temp_dict['-41'][0][0])
            stat_info_dict['Pred_-41_mean'].append(temp_dict['-41'][0][1])
            stat_info_dict['Pred_-41_std'].append(temp_dict['-41'][0][2])
                
                
                
                
            
    ############# after finishing, let's save it
   
    stat_info_df = pd.DataFrame.from_dict(stat_info_dict)
    stat_info_file = 'stat_results/stat_info.csv'
    if not os.path.exists('stat_results'):
        os.makedirs('stat_results')
    else:
        pass
    stat_info_df.to_csv(stat_info_file, index=False)
    
    print('')
    print('Congrats! The statistical information about predictions has been saved!')
            
            

[-29.5        -12.50199986 -41.90000153]
[-41.90000153 -29.5        -12.50199986]
[-29.5        -41.90000153 -12.50199986]

Congrats! The statistical information about predictions has been saved!
